<a href="https://colab.research.google.com/github/patjiang/openBind/blob/main/fileformatter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title set up condacolab
%%time
import os, time, signal
import sys, random, string, re
os.system("pip install -q condacolab")
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!
CPU times: user 7.32 ms, sys: 866 µs, total: 8.18 ms
Wall time: 918 ms


In [ ]:
%%time
import os, subprocess
os.system("conda install mdtraj")
os.system("conda install parmed")
os.system("conda install rdkit")
os.system("conda install yaml")
os.system("pip install pyyaml")
os.system("conda install easydict")
os.system("pip install py3Dmol")
os.system("pip install python-pdb")
os.system("pip install biopandas")
os.system("conda install -c openbabel openbabel")
subprocess.run("mamba install openmm=7.7.0 pdbfixer -y", shell=True)
subprocess.run("mamba install openmmforcefields -c conda-forge -y", shell=True)
os.system("pip install biopython")

CPU times: user 1.49 s, sys: 188 ms, total: 1.68 s
Wall time: 5min 12s


0

In [ ]:
#@title Split the chains of input pdb, define bounding boxes.
import py3Dmol
from Bio.PDB import PDBParser
from Bio.PDB.PDBIO import PDBIO
from google.colab import files
import json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
from python_pdb.parsers import parse_pdb_to_pandas
from openbabel import openbabel
import csv

def get_pdb(pdb_code=None):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  elif os.path.isfile(pdb_code):
    return pdb_code
  elif len(pdb_code) == 4:
    if not os.path.isfile(f"{pdb_code}.pdb1"):
      os.system(f"wget -qnc https://files.rcsb.org/download/{pdb_code}.pdb1.gz")
      os.system(f"gunzip {pdb_code}.pdb1.gz")
    return f"{pdb_code}.pdb1"
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
    return f"AF-{pdb_code}-F1-model_v3.pdb"

def multiFileHandler(tdict, key, idx:int):
  upload_dict = tdict
  pdb_string = upload_dict[key]
  with open(f"tmp{idx}.pdb","wb") as out: out.write(pdb_string)
  return f"tmp{idx}.pdb"


def nearest_residues(pdbDF, box, num):
  reslist = []
  add_thresh = 0
  df1 = pdbDF.sort_values('pos_x')
  xthresh = add_thresh + box[3]
  df2 = df1.loc[(df1['pos_x'] > (box[0] - xthresh)) & (df1['pos_x'] < (box[0] + xthresh))]
  #print(df2.size)

  df3 = df2.sort_values('pos_y')
  ythresh = add_thresh + box[4]
  df4 = df3.loc[(df3['pos_y'] > (box[1] - ythresh)) & (df3['pos_y'] < (box[1] + ythresh))]
  #print(df4.size)

  df5 = df4.sort_values('pos_z')
  zthresh = add_thresh + box[5]
  df6 = df5.loc[(df5['pos_z'] > (box[2] - zthresh)) & (df5['pos_z'] < (box[2] + zthresh))]
  #print(df6.size)


  unique_residues = set(df6['residue_seq_id'])
  for i in unique_residues:
    toadd = f"{list(df6.loc[df6['residue_seq_id'] == i]['chain_id'])[0]}_{i}_{list(df6.loc[df6['residue_seq_id'] == i]['residue_name'])[0]}"
    reslist.append(toadd)

  return reslist

def get_box(lf, pf, resnum = 10):
  with open(lf, 'r') as f:
    ldf = parse_pdb_to_pandas(f.read())

  minX = min(ldf["pos_x"])
  minY = min(ldf["pos_y"])
  minZ = min(ldf["pos_z"])

  maxX = max(ldf["pos_x"])
  maxY = max(ldf["pos_y"])
  maxZ = max(ldf["pos_z"])
  box = [((maxX + minX)/2), ((maxY + minY)/2), ((maxZ + minZ)/2), maxX-minX, maxY-minY, maxZ-minZ]

  with open(pf, 'r') as f:
    pdf = parse_pdb_to_pandas(f.read())

  residues  = nearest_residues(pdf, box, resnum)

  return box, residues



def convResults(dir, path, parts):
  parser = PDBParser()
  io = PDBIO()
  updict = files.upload()
  keys = np.sort(list(updict.keys()))
  i = 0
  pnum = 1
  for k in keys:
    if parts:
      tfname = multiFileHandler(updict, k, i)
      structure = parser.get_structure(path,tfname)
      pdb_chains = structure.get_chains()
      index = tfname.split("tmp")[1].split(".")[0]
      if(int(index) == 0):
        pnum = 1
        os.system(f"mkdir {dir}/part{pnum}")
      elif((int(index))%64 == 0):
        pnum += 1
        os.system(f"mkdir {dir}/part{pnum}")
      os.system(f"mkdir {dir}/part{pnum}/{path}{index}")

      for chain in pdb_chains:
          io.set_structure(chain)
          fil = str(structure.get_id()) + "_" + str(chain.get_id()) + ".pdb"
          io.save(f"{dir}/part{pnum}/{path}{index}/{fil}")

      box, residues = get_box(f"{dir}/part{pnum}/{path}{index}/{path}_B.pdb", f"{dir}/part{pnum}/{path}{index}/{path}_A.pdb")

      with open(f'{dir}/part{pnum}/{path}{index}/box.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(box)
        file.close()

      with open(f'{dir}/part{pnum}/{path}{index}/pocket.txt', 'w', newline='') as file:
        for aa in residues:
          file.write(f"{aa}\n")
        file.close()

      os.system(f"obabel /content/{dir}/part{pnum}/{path}{index}/{path}_B.pdb -O /content/{dir}/part{pnum}/{path}{index}/ligand.sdf")
      os.system(f"obabel /content/{dir}/part{pnum}/{path}{index}/{path}_B.pdb -O /content/{dir}/part{pnum}/{path}{index}/ligand.mol2")
      os.system(f"obabel /content/{dir}/part{pnum}/{path}{index}/{path}_B.pdb -O temp.smi")
      smiles = open(f"temp.smi").read().split("	")[0]
      #print(smiles)
      open(f"/content/{dir}/part{pnum}/{path}{index}/ligand.txt", "w").write(smiles)
      #os.system(f"rm out/{path}/{path}_B.pdb")
      os.rename(f"/content/{dir}/part{pnum}/{path}{index}/{path}_A.pdb", f"/content/{dir}/part{pnum}/{path}{index}/protein.pdb")
      os.rename(f"/content/{dir}/part{pnum}/{path}{index}/{path}_B.pdb", f"/content/{dir}/part{pnum}/{path}{index}/ligand.pdb")
      i += 1
    else:
      tfname = multiFileHandler(updict, k, i)
      structure = parser.get_structure(path,tfname)
      pdb_chains = structure.get_chains()
      index = tfname.split("tmp")[1].split(".")[0]
      os.system(f"mkdir {dir}/{path}{index}")
      for chain in pdb_chains:
          io.set_structure(chain)
          fil = str(structure.get_id()) + "_" + str(chain.get_id()) + ".pdb"
          io.save(f"{dir}/{path}{index}/{fil}")

      box, residues = get_box(f"{dir}/{path}{index}/{path}_B.pdb", f"{dir}/{path}{index}/{path}_A.pdb")

      with open(f'{dir}/{path}{index}/box.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(box)
        file.close()

      with open(f'{dir}/{path}{index}/pocket.txt', 'w', newline='') as file:
        for aa in residues:
          file.write(f"{aa}\n")
        file.close()

      os.system(f"obabel /content/{dir}/{path}{index}/{path}_B.pdb -O /content/{dir}/{path}{index}/ligand.sdf")
      os.system(f"obabel /content/{dir}/{path}{index}/{path}_B.pdb -O /content/{dir}/{path}{index}/ligand.mol2")
      os.system(f"obabel /content/{dir}/{path}{index}/{path}_B.pdb -O temp.smi")
      smiles = open(f"temp.smi").read().split("	")[0]
      #print(smiles)
      open(f"/content/{dir}/{path}{index}/ligand.txt", "w").write(smiles)
      #os.system(f"rm out/{path}/{path}_B.pdb")
      os.rename(f"/content/{dir}/{path}{index}/{path}_A.pdb", f"/content/{dir}/{path}{index}/protein.pdb")
      os.rename(f"/content/{dir}/{path}{index}/{path}_B.pdb", f"/content/{dir}/{path}{index}/ligand.pdb")
      i += 1

In [ ]:
import warnings
warnings.filterwarnings('ignore')
dir = "iter20" #@param {type:"string"}
path = "sys" #@param {type:"string"}
parts = True #@param {type: "boolean"}
#multiple_files = 1 #@param {type: "integer"}

os.system(f"mkdir /content/{dir}")
#for i in range(multiple_files):
#  j = i + 1
#  np = path + str(j)
convResults(dir, path, parts)

Saving design31_n15.pdb to design31_n15 (2).pdb
Saving design31_n14.pdb to design31_n14 (2).pdb
Saving design31_n13.pdb to design31_n13 (2).pdb
Saving design31_n12.pdb to design31_n12 (2).pdb
Saving design31_n11.pdb to design31_n11 (2).pdb
Saving design31_n10.pdb to design31_n10 (2).pdb
Saving design31_n9.pdb to design31_n9 (2).pdb
Saving design31_n8.pdb to design31_n8 (2).pdb
Saving design31_n7.pdb to design31_n7 (2).pdb
Saving design31_n6.pdb to design31_n6 (2).pdb
Saving design31_n5.pdb to design31_n5 (2).pdb
Saving design31_n4.pdb to design31_n4 (2).pdb
Saving design31_n3.pdb to design31_n3 (2).pdb
Saving design31_n2.pdb to design31_n2 (2).pdb
Saving design31_n1.pdb to design31_n1 (2).pdb
Saving design31_n0.pdb to design31_n0 (2).pdb
Saving design30_n15.pdb to design30_n15 (2).pdb
Saving design30_n14.pdb to design30_n14 (2).pdb
Saving design30_n13.pdb to design30_n13 (2).pdb
Saving design30_n12.pdb to design30_n12 (2).pdb
Saving design30_n11.pdb to design30_n11 (2).pdb
Saving desig

In [ ]:
#@title display box and results
from rdkit import Chem
from rdkit.Chem import AllChem
def showResults(dir, finform = "sdf"):
  def smi2conf(smiles):
    '''Convert SMILES to rdkit.Mol with 3D coordinates'''
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol, useRandomCoords=True)
        rdmolops.RemoveStereochemistry(mol)
        AllChem.MMFFOptimizeMolecule(mol, maxIters=200)
        return mol
    else:
        return None

  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
  pdb_str = open(f"/content/{dir}/protein.pdb",'r').read()
  view.addModel(pdb_str,'pdb')

  with open(f"/content/iter18/sys0/box.csv", 'r') as csvFile:
      reader = csv.reader(csvFile)
      vals = [float(x) for x in list(reader)[0]]

  if finform == "sdf":
    lig_str = open(f'/content/{dir}/ligand.sdf','r').read()
    view.addModel(lig_str,'sdf')

  elif finform == "pdb":
    lig_str = open(f'/content/{dir}/ligand.pdb','r').read()
    view.addModel(lig_str,'pdb')

  elif finform == "smi":
    lig_str = open(f'/content/{dir}/ligand.txt','r').read()
    ligMol = smi2conf(lig_str)
    mblock = Chem.MolToMolBlock(ligMol)
    view.addModel(mblock, 'mol')



  view.setStyle({"model":0},{'cartoon':{'color': 'spectrum'}})
  view.setStyle({"model":1},{'stick':{'color': 'spectrum'}})
  view.addBox({'center':{'x':vals[0],'y': vals[1],'z': vals[2]},'dimensions': {'w':vals[3],'h':vals[4],'d':vals[5]}, 'color': 'yellow', "opacity": 0.4});
  #print(vals[0],vals[1], vals[2], vals[3], vals[4], vals[5])
  view.addSurface(py3Dmol.VDW,{'opacity':0.7,'color':'spectrum'}, {"model":1})
  view.zoomTo()
  return view


showResults(dir = "iter18/sys0", finform = "smi")

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, rdmolops

def MolTo3DView(mol, size=(300, 300), style="stick", surface=False, opacity=0.5):
    """Draw molecule in 3D

    Args:
    ----
        mol: rdMol, molecule to show
        size: tuple(int, int), canvas size
        style: str, type of drawing molecule
               style can be 'line', 'stick', 'sphere', 'cartoon'
        surface, bool, display SAS
        opacity, float, opacity of surface, range 0.0-1.0
    Return:
    ----
        viewer: py3Dmol.view, a class for constructing embedded 3Dmol.js views in ipython notebooks.
    """
    assert style in ('line', 'stick', 'sphere', 'cartoon')
    mblock = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=size[0], height=size[1])
    viewer.addModel(mblock, 'mol')
    #viewer.setStyle({style:{'color': 'spectrum'}})
    if surface:
        viewer.addSurface(py3Dmol.VDW, {'opacity': opacity})
    #viewer.zoomTo()
    return viewer



def smi2conf(smiles):
    '''Convert SMILES to rdkit.Mol with 3D coordinates'''
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol, useRandomCoords=True)
        rdmolops.RemoveStereochemistry(mol)
        AllChem.MMFFOptimizeMolecule(mol, maxIters=200)
        return mol
    else:
        return None




smi = open(f'/content/iter18/sys0/ligand.txt','r').read()
conf = smi2conf(smi)
viewer = MolTo3DView(conf, size=(600, 300), style='line', surface = True)
viewer.setStyle({"model":0},{'stick':{'color': 'spectrum'}})

lig_str = open(f'/content/iter18/sys0/ligand.sdf','r').read()
viewer.addModel(lig_str,'sdf')
viewer.addSurface(py3Dmol.VDW,{'opacity':0.7,'color':'spectrum'}, {"model":1})

lig_str = open(f'/content/iter18/sys0/ligand.pdb','r').read()
viewer.addModel(lig_str,'pdb')
viewer.setStyle({"model":2},{'stick':{'color': 'yellow'}})


#molstr = Chem.MolFromMol2File('/content/iter18/sys0/ligand.mol2')
#if molstr is not None:
#  print(type(molstr))
#  molstr = Chem.AddHs(molstr)
#  molblock = Chem.MolToMolBlock(molstr)
#  viewer.addModel(molblock, 'mol')
#  viewer.setStyle({"model":2},{'stick':{'color': 'yellow'}})
#else:
#  print("Failed to read mol2 file")



viewer.show()

In [ ]:
!zip -r batch4.zip /content/iter19 /content/iter20
files.download(f"batch4.zip")

Streaming output truncated to the last 5000 lines.
  adding: content/iter19/part7/sys404/ligand.mol2 (deflated 72%)
  adding: content/iter19/part7/sys404/ligand.pdb (deflated 76%)
  adding: content/iter19/part7/sys404/box.csv (deflated 18%)
  adding: content/iter19/part7/sys404/protein.pdb (deflated 78%)
  adding: content/iter19/part7/sys404/pocket.txt (deflated 63%)
  adding: content/iter19/part7/sys404/ligand.txt (deflated 66%)
  adding: content/iter19/part7/sys401/ (stored 0%)
  adding: content/iter19/part7/sys401/ligand.sdf (deflated 80%)
  adding: content/iter19/part7/sys401/ligand.mol2 (deflated 72%)
  adding: content/iter19/part7/sys401/ligand.pdb (deflated 76%)
  adding: content/iter19/part7/sys401/box.csv (deflated 40%)
  adding: content/iter19/part7/sys401/protein.pdb (deflated 78%)
  adding: content/iter19/part7/sys401/pocket.txt (deflated 63%)
  adding: content/iter19/part7/sys401/ligand.txt (deflated 67%)
  adding: content/iter19/part7/sys397/ (stored 0%)
  adding: content

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
parser = PDBParser()
io = PDBIO()
tfname = "4g1q.pdb"
structure = parser.get_structure("sys", tfname)
pdb_chains = structure.get_chains()
index = tfname.split(".")[0]
os.system(f"mkdir {index}")
for chain in pdb_chains:
    io.set_structure(chain)
    fil = str(structure.get_id()) + "_" + str(chain.get_id()) + ".pdb"
    io.save(f"{index}/{fil}")

/usr/local/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 24554.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 24686.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 24730.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 25750.
  warnings.warn(


In [ ]:
with open("4g1q-trim.pdb", "w") as fil:
  with open("/content/4g1q/sys_A.pdb", "r") as a:
    afile = a.read()
  with open("/content/4g1q/sys_B.pdb", "r") as b:
    bfile = b.read()
  fil.write(afile)
  fil.write(bfile)

with open("4g1q-trim.pdb", "r") as test:
  print(test.read())


ATOM      1  N   MET A  -1      72.780 -28.892  66.585  1.00 46.46           N  
ATOM      2  CA  MET A  -1      71.717 -28.078  65.924  1.00 49.12           C  
ATOM      3  C   MET A  -1      72.241 -27.408  64.639  1.00 44.50           C  
ATOM      4  O   MET A  -1      73.327 -26.837  64.615  1.00 43.57           O  
ATOM      5  CB  MET A  -1      71.244 -27.033  66.934  1.00 56.97           C  
ATOM      6  CG  MET A  -1      70.758 -25.757  66.361  1.00 63.65           C  
ATOM      7  SD  MET A  -1      69.000 -25.805  66.052  1.00 70.05           S  
ATOM      8  CE  MET A  -1      68.735 -24.088  65.587  1.00 70.68           C  
ATOM      9  H1  MET A  -1      72.437 -29.264  67.401  1.00 55.75           H  
ATOM     10  H2  MET A  -1      73.050 -29.602  65.998  1.00 55.75           H  
ATOM     11  H3  MET A  -1      73.536 -28.334  66.780  1.00 55.75           H  
ATOM     12  HA  MET A  -1      70.958 -28.656  65.701  1.00 58.95           H  
ATOM     13  HB2 MET A  -1  